# Controlling a submarine

- https://adventofcode.com/2021/day/2

Time to figure out how to dive with a submarine. The first task is a common one: interpret instructions that map out the submarine path.


In [1]:
from __future__ import annotations

from dataclasses import dataclass, replace
from enum import Enum
from functools import reduce
from typing import Iterable


class SubmarineDirection(Enum):
    forward = (1, 0)
    down = (0, 1)
    up = (0, -1)


@dataclass
class SubmarineMove:
    direction: SubmarineDirection
    dpos: int = 0
    ddepth: int = 0

    @classmethod
    def from_line(cls, line: str) -> SubmarineMove:
        dir, amount = line.split()
        direction = SubmarineDirection[dir]
        return cls(direction, *direction.value) * int(amount)

    def __mul__(self, amount: int) -> SubmarineMove:
        return replace(self, dpos=self.dpos * amount, ddepth=self.ddepth * amount)


@dataclass
class SubmarinePosition:
    position: int = 0
    depth: int = 0

    def change_position(self, move: SubmarineMove) -> SubmarinePosition:
        return replace(
            self, position=self.position + move.dpos, depth=self.depth + move.ddepth
        )

    @classmethod
    def from_moves(cls, moves: Iterable) -> SubmarinePosition:
        return reduce(cls.change_position, moves, cls())


test_moves = [
    SubmarineMove.from_line(line)
    for line in """\
forward 5
down 5
forward 8
up 3
down 8
forward 2
""".splitlines()
]

test_pos = SubmarinePosition.from_moves(test_moves)
assert test_pos.position == 15
assert test_pos.depth == 10
assert test_pos.position * test_pos.depth == 150

In [2]:
import aocd

moves = [
    SubmarineMove.from_line(line)
    for line in aocd.get_data(day=2, year=2021).splitlines()
]
submarine_pos = SubmarinePosition.from_moves(moves)
print("Part 1:", submarine_pos.depth * submarine_pos.position)

Part 1: 2039912


# Part 2: reinterpreting the directions

Now, instead of a simple 2-direction vector problem, we have a slightly more complicated set of moves. The way that the submarine depth changes now depends on the `aim` value, and the `up` and `down` commands only affect the aim.

Rather than re-create the `SubmarineMove` class only to rename `ddepth` (delta depth) to `daim` (delta aim), I'm just going to reinterpret the `ddepth` value as delta aim here. Welcome to Technical Debt, 101! :-D

It means we only have to provide a new `SubmarinePosition` implementation to achieve part 2.


In [3]:
@dataclass
class AimedSubmarinePosition(SubmarinePosition):
    aim: int = 0

    def change_position(self, move: SubmarineMove) -> AimedSubmarinePosition:
        return replace(
            self,
            position=self.position + move.dpos,
            depth=self.depth + (self.aim * move.dpos),
            aim=self.aim + move.ddepth,  # delta depth is really delta aim
        )


test_pos = AimedSubmarinePosition.from_moves(test_moves)
assert test_pos.position == 15
assert test_pos.depth == 60
assert test_pos.position * test_pos.depth == 900

In [4]:
submarine_pos = AimedSubmarinePosition.from_moves(moves)
print("Part 2:", submarine_pos.depth * submarine_pos.position)

Part 2: 1942068080
